In [ ]:
import os
import pandas as pd
import numpy as np
from sklearn.svm import SVR
from sklearn.model_selection import RandomizedSearchCV
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.inspection import permutation_importance
from scipy.stats import loguniform
import matplotlib.pyplot as plt
from save_and_compare_results import *
import shap


In [ ]:
# Define the path to the parent directory
data_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))  # Move one level up

X = pd.read_pickle(os.path.join(data_dir, "data/X_matrix_proteins_nb.pkl"))
Y = pd.read_pickle(os.path.join(data_dir, "data/Y_matrix_proteins_nb.pkl"))

# Preprocess the data
X = X.drop(columns=["Yeast_ID"]).fillna(0)
Y = Y.drop(columns=["Yeast_ID"]).fillna(Y.drop(columns=["Yeast_ID"]).mean())

In [4]:
param_distributions = {
    "C": loguniform(1e-3, 1e3),
    "epsilon": loguniform(1e-3, 1),
    "gamma": ["scale", "auto"],
    "kernel": ["linear", "rbf"]
}

svr = SVR()
random_search = RandomizedSearchCV(
    estimator=svr, 
    param_distributions=param_distributions,
    n_iter=50, 
    cv=3,
    verbose=1,
    random_state=42,
    n_jobs=-1
) 

random_search.fit(X, Y)

Fitting 3 folds for each of 50 candidates, totalling 150 fits


/Users/claradelandre/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/claradelandre/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/claradelandre/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/claradelandre/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was pass

RandomizedSearchCV(cv=3, estimator=SVR(), n_iter=50, n_jobs=-1,
                   param_distributions={'C': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fda01230580>,
                                        'epsilon': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fda0122d0d0>,
                                        'gamma': ['scale', 'auto'],
                                        'kernel': ['linear', 'rbf']},
                   random_state=42, verbose=1)

In [5]:
# Get the best model and parameters
best_svm = random_search.best_estimator_
best_params = random_search.best_params_
print("\nBest hyperparameters:", best_params)


Best hyperparameters: {'C': 0.21481457181982688, 'epsilon': 0.0065169906111771725, 'gamma': 'scale', 'kernel': 'rbf'}


In [6]:
# Evaluate the best model
y_pred = best_svm.predict(X)
r2 = r2_score(Y, y_pred)
mse = mean_squared_error(Y, y_pred)
print(f"\nR² Score: {r2:.4f}, Mean Squared Error: {mse:.4f}")


R² Score: 0.2517, Mean Squared Error: 0.0375


With number :  
best hyperparameters: {'C': 0.21481457181982688, 'epsilon': 0.0065169906111771725, 'gamma': 'scale', 'kernel': 'rbf'}  
R² Score: 0.2517, Mean Squared Error: 0.0375  

# Model features importance

In [8]:
if best_params["kernel"] == "linear":
    print("Calculating feature importance for linear kernel...")
    
    # Feature importance is directly derived from the coefficients
    feature_importances = np.abs(best_svm.coef_[0])
    
    # Save and visualize feature importance
    save_feature_importance(
        features=X.columns,
        importance_scores=feature_importances,
        method="Coefficients",
        model_name="SVR"
    )
    
    # Display and plot top features
    feature_importances_df = pd.DataFrame({
        "Feature": X.columns,
        "Importance": feature_importances
    }).sort_values(by="Importance", ascending=False)
    
    top_features = feature_importances_df.head(10)
    print("\nTop 10 Features (Linear Kernel):")
    print(top_features)
    
    plt.figure(figsize=(10, 6))
    plt.barh(top_features["Feature"], top_features["Importance"], color="skyblue")
    plt.xlabel("Coefficient Magnitude")
    plt.ylabel("Feature")
    plt.title("Top 10 Features Impacting YPD Doubling Time (Linear Kernel)")
    plt.gca().invert_yaxis()
    plt.show()

In [9]:
# Afficher et tracer les 10 caractéristiques les plus importantes
print("Affichage des 10 caractéristiques les plus importantes...")
feature_importances_df = pd.DataFrame({
    "Feature": X.columns,
    "Importance": best_svm.feature_importances_
}).sort_values(by="Importance", ascending=False)

# Sélectionner les 10 caractéristiques les plus importantes
top_mutations = feature_importances_df.head(10)

# Afficher dans la console
print("\nMutations ayant le plus d'impact sur le YPD doubling time :")
print(top_mutations)

# Créer le graphique
plt.figure(figsize=(10, 6))
plt.barh(top_mutations["Feature"], top_mutations["Importance"], color="skyblue")
plt.xlabel("Importance")
plt.ylabel("Mutation")
plt.title("Top 10 Mutations Impacting YPD Doubling Time")
plt.gca().invert_yaxis()
plt.show()

Affichage des 10 caractéristiques les plus importantes...


AttributeError: 'SVR' object has no attribute 'feature_importances_'

# SHAP features importance

In [10]:
print("Calculating SHAP values for SVR...")
explainer = shap.KernelExplainer(best_svm.predict, X)

# Calculate SHAP values
shap_values = explainer.shap_values(X, nsamples=100)

# Compute mean absolute SHAP values for feature importance
shap_mean_importance = np.abs(shap_values).mean(axis=0)

# Save SHAP feature importance
save_feature_importance(
    features=X.columns,
    importance_scores=shap_mean_importance,
    method="SHAP",
    model_name="SVR"
)

# Display and plot SHAP importance
feature_importances_df = pd.DataFrame({
    "Feature": X.columns,
    "Importance": shap_mean_importance
}).sort_values(by="Importance", ascending=False)

top_features = feature_importances_df.head(10)
print("\nTop 10 Features (SHAP):")
print(top_features)

plt.figure(figsize=(10, 6))
plt.barh(top_features["Feature"], top_features["Importance"], color="skyblue")
plt.xlabel("SHAP Value (Mean Absolute Importance)")
plt.ylabel("Feature")
plt.title("Top 10 Features Impacting YPD Doubling Time (SHAP)")
plt.gca().invert_yaxis()
plt.show()

Calculating SHAP values for SVR...


Using 792 background data samples could cause slower run times. Consider using shap.sample(data, K) or shap.kmeans(data, K) to summarize the background as K samples.


  0%|          | 0/792 [00:00<?, ?it/s]

/Users/claradelandre/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_least_angle.py:682: ConvergenceWarning: Early stopping the lars path, as the residues are small and the current value of alpha is no longer well controlled. 52 iterations, alpha=1.173e-02, previous alpha=1.173e-02, with an active set of 19 regressors.
  warnings.warn(


ValueError: You are using LassoLarsIC in the case where the number of samples is smaller than the number of features. In this setting, getting a good estimate for the variance of the noise is not possible. Provide an estimate of the noise variance in the constructor.

In [ ]:
print("Generating SHAP plots...")
shap.summary_plot(shap_values, X, plot_type="bar")
shap.summary_plot(shap_values, X)

# Permutation Importance

In [ ]:
print("Calculating permutation importance...")
perm_importance = permutation_importance(best_svm, X, Y.values.ravel(), n_repeats=10, random_state=42, scoring="r2")
feature_importances = perm_importance.importances_mean

# Save and visualize permutation importance
save_feature_importance(
    features=X.columns,
    importance_scores=feature_importances,
    method="Permutation",
    model_name="SVR"
)

# Display and plot permutation importance
feature_importances_df = pd.DataFrame({
    "Feature": X.columns,
    "Importance": feature_importances
}).sort_values(by="Importance", ascending=False)

top_features = feature_importances_df.head(10)
print("\nTop 10 Features (Permutation Importance):")
print(top_features)

plt.figure(figsize=(10, 6))
plt.barh(top_features["Feature"], top_features["Importance"], color="skyblue")
plt.xlabel("Permutation Importance (Mean)")
plt.ylabel("Feature")
plt.title("Top 10 Features Impacting YPD Doubling Time (Permutation Importance)")
plt.gca().invert_yaxis()
plt.show()